# データの読み込み

In [1]:
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
import torchvision.transforms as transforms
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import skorch
import pandas as pd
import sklearn
import csv
import os

In [2]:
from __future__ import print_function
from torch.utils import data
from models import *
from utils import Visualizer, view_model
import random
import time
from config.config import Config
from torch.nn import DataParallel
from torch.optim.lr_scheduler import StepLR
from test import *

In [3]:
DATA_FOLDER = '/home/tamaru/scene_categorize/main/data/'
csv_path = os.path.join(DATA_FOLDER, 'resized_data.csv')
datalist = pd.read_csv(csv_path, names=["img_path", "l_class", 's_class'])
datalist.head()

,img_path,l_class,s_class
0,/home/tamaru/scene_categorize/main/data/insta_...,5gokan-denkisogokan,5gokan-denkisogokan
1,/home/tamaru/scene_categorize/main/data/insta_...,5gokan-denkisogokan,5gokan-denkisogokan
2,/home/tamaru/scene_categorize/main/data/insta_...,5gokan-denkisogokan,5gokan-denkisogokan
3,/home/tamaru/scene_categorize/main/data/insta_...,5gokan-denkisogokan,5gokan-denkisogokan
4,/home/tamaru/scene_categorize/main/data/insta_...,5gokan-denkisogokan,5gokan-denkisogokan


#insta_frames->insta_cubemap datalist
c_np = np.array(datalist)
col_name = datalist.columns.values
for row in c_np:
    row[0] = row[0].replace('insta_frames','insta_cubemap')
c_df = pd.DataFrame(c_np, columns=col_name)
c_df.to_csv('../data/cubemap_data.csv', index=False)

In [4]:
dfs = datalist.drop(['l_class'], axis=1)
dfs.groupby('s_class').count()

,img_path
s_class,
4gokan_inside,42
4gokan_outside_east,18
4gokan_outside_west,248
4gokan_stairs,750
5gokan-denkisogokan,361
5gokan-mediacenter,127
5gokan_1F,65
5gokan_2F,96
5gokan_3F,32


In [5]:
dfs.img_path.count()

8930

In [6]:
heatmap_df = pd.read_csv("/home/tamaru/scene_categorize/main/data/inpainting_data.csv", names=["img_path", "l_class", 's_class'])
heatmap_df = heatmap_df.drop(['l_class'], axis=1)
heatmap_df

,img_path,s_class
0,/home/tamaru/scene_categorize/main/data/inpain...,5gokan-denkisogokan
1,/home/tamaru/scene_categorize/main/data/inpain...,8gokan_ent_north
2,/home/tamaru/scene_categorize/main/data/inpain...,8gokan_1F
3,/home/tamaru/scene_categorize/main/data/inpain...,8gokan_1F
4,/home/tamaru/scene_categorize/main/data/inpain...,8gokan_stairs_cnt
5,/home/tamaru/scene_categorize/main/data/inpain...,denkisogokan_4F
6,/home/tamaru/scene_categorize/main/data/inpain...,denkisogokan_lounge
7,/home/tamaru/scene_categorize/main/data/inpain...,denkisogokan_2F
8,/home/tamaru/scene_categorize/main/data/inpain...,denkisogokan_3F
9,/home/tamaru/scene_categorize/main/data/inpain...,lounge


labs = dfs[dfs['s_class'].str.startswith('lab')]
num_labs_class = 10
labs.s_class.unique()


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
le = LabelEncoder() 
le.fit(dfs.s_class) 
dfs["labels"] = le.transform(dfs.s_class) 
dfs.groupby('labels')

In [9]:
cor_table = dfs.groupby('labels').s_class.unique() 
cor_table = pd.DataFrame(cor_table) 
num_s_class = len(cor_table)
cor_table

,s_class
labels,
0,[4gokan_inside]
1,[4gokan_outside_east]
2,[4gokan_outside_west]
3,[4gokan_stairs]
4,[5gokan-denkisogokan]
5,[5gokan-mediacenter]
6,[5gokan_1F]
7,[5gokan_2F]
8,[5gokan_3F]


In [10]:
heatmap_df['labels'] = le.transform(heatmap_df.s_class)
heatmap_table = heatmap_df.groupby('labels').s_class.unique()
heatmap_df= heatmap_df.drop(['s_class'], axis=1)
heatmap_table

labels
0           [4gokan_inside]
1     [4gokan_outside_east]
2     [4gokan_outside_west]
3           [4gokan_stairs]
4     [5gokan-denkisogokan]
5      [5gokan-mediacenter]
6               [5gokan_1F]
9         [5gokan_ent_east]
10          [5gokan_lounge]
11        [5gokan_out_east]
12         [5gokan_parking]
13         [5gokan_smoking]
14     [5gokan_stairs_east]
16              [8gokan_1F]
17       [8gokan_ent_north]
19      [8gokan_stairs_cnt]
21        [denkisogokan_2F]
22        [denkisogokan_3F]
23        [denkisogokan_4F]
25    [denkisogokan_lounge]
27             [lab_bs_cnt]
28             [lab_corner]
29               [lab_desk]
30                [lab_ent]
31            [lab_printer]
32              [lab_table]
33             [lab_wb_cnt]
34             [lab_wb_ent]
35                 [lounge]
Name: s_class, dtype: object

In [11]:
dfs = dfs.drop(['s_class'], axis=1)
dfs

,img_path,labels
0,/home/tamaru/scene_categorize/main/data/insta_...,4
1,/home/tamaru/scene_categorize/main/data/insta_...,4
2,/home/tamaru/scene_categorize/main/data/insta_...,4
3,/home/tamaru/scene_categorize/main/data/insta_...,4
4,/home/tamaru/scene_categorize/main/data/insta_...,4
5,/home/tamaru/scene_categorize/main/data/insta_...,4
6,/home/tamaru/scene_categorize/main/data/insta_...,4
7,/home/tamaru/scene_categorize/main/data/insta_...,4
8,/home/tamaru/scene_categorize/main/data/insta_...,4
9,/home/tamaru/scene_categorize/main/data/insta_...,4


# trainデータ, testデータの分割

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
train_data, test_data = train_test_split(dfs, test_size=0.2, random_state=42, stratify=dfs.labels)

In [14]:
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42, stratify=train_data.labels)

In [15]:
#画像の前処理を定義
data_transforms = {
    'data': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ])
}
#正規化をしない処理
to_tensor_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor()
])

In [16]:
class CustomDataset(torch.utils.data.Dataset):
        
    def __init__(self, dataframe, root_dir, transform=None):
        #前処理クラスの指定
        self.transform = transform
        #pandasでcsvデータの読み出し
        #画像とラベルの一覧を保持するリスト
        self.images = np.array(dataframe.img_path).tolist()
        self.labels = np.array(dataframe.labels).tolist()
        self.root_dir = root_dir
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        #dataframeから画像へのパスとラベルを読み出す
        label = self.labels[idx]
        img = self.images[idx]
        #画像の読み込み
        with open(img, 'rb') as f:
            image = Image.open(f)
            image = image.convert('RGB')
            image = image.resize((224,224))
        #画像への処理
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

In [17]:
train_set = CustomDataset(dataframe=train_data, root_dir="../data/insta_frames", transform=data_transforms['data'])
val_set = CustomDataset(dataframe=val_data, root_dir="../data/insta_frames", transform=data_transforms['data'])
test_set = CustomDataset(dataframe=test_data, root_dir="../data/insta_frames", transform=data_transforms['data'])

In [18]:
# DataLoaderのcollate_fnはバッチ内のtensorのshapeをすべて同じにする必要がある
# 自分で指定してエラーが起きないようにする
def my_collate_fn(batch):
    # datasetの出力が
    # [image, target] = dataset[batch_idx]
    # の場合.
    images = []
    labels = []
    for image, label in batch:
        images.append(image)
        labels.append(label)
    images = torch.stack(images,dim=0)
    return images, labels

In [19]:
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=16, shuffle=True, num_workers=6)
val_loader = torch.utils.data.DataLoader(dataset=val_set, batch_size=16, shuffle=False, num_workers=6)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=16, shuffle=False, num_workers=6)

In [20]:
heatmap_set = CustomDataset(dataframe=heatmap_df, root_dir="../data/inpainting_data", transform=data_transforms['data'])
heatmap_loader = torch.utils.data.DataLoader(dataset=heatmap_set, batch_size=1, shuffle=False, num_workers=6)

# ネットワークの定義

In [21]:
import torchvision.models as models
import bagnets.pytorchnet

In [22]:
num_classes = num_s_class
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = bagnets.pytorchnet.bagnet17(pretrained=True)

In [23]:
from models.focal_loss import *

In [24]:
num_ftrs = net.fc.in_features

In [27]:
# #パラメータ凍結と採取層クラス数変更
# for param in net.parameters():
#     param.requires_grad = False
#最終層をnum_s_classクラス用に変更
opt = Config()
# net.avgpool = None
net.fc = nn.Sequential(*[nn.BatchNorm2d(num_ftrs), nn.Dropout(), nn.Linear(num_ftrs, 512), nn.BatchNorm1d(512)])

net.relu = nn.PReLU()
net.layer1[0].relu = nn.PReLU()
net.layer1[1].relu = nn.PReLU()
net.layer1[2].relu = nn.PReLU()
net.layer2[0].relu = nn.PReLU()
net.layer2[1].relu = nn.PReLU()
net.layer2[2].relu = nn.PReLU()
net.layer2[3].relu = nn.PReLU()
net.layer3[0].relu = nn.PReLU()
net.layer3[1].relu = nn.PReLU()
net.layer3[2].relu = nn.PReLU()
net.layer3[3].relu = nn.PReLU()
net.layer3[4].relu = nn.PReLU()
net.layer3[5].relu = nn.PReLU()
net.layer4[0].relu = nn.PReLU()
net.layer4[1].relu = nn.PReLU()
net.layer4[2].relu = nn.PReLU()
net.avgpool = None
metric_fc = ArcMarginProduct(512, num_classes, s=10, m=0.1, easy_margin=opt.easy_margin)
#最適化関数
criterion = FocalLoss(gamma=2)
optimizer = optim.SGD([{'params': net.parameters()}, {'params': metric_fc.parameters()}],lr=opt.lr, weight_decay=opt.weight_decay)
scheduler = StepLR(optimizer, step_size=opt.lr_step, gamma=0.1)
net = net.to(device)
metric_fc.to(device)
print(net)

BagNet(
  (conv1): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.001, affine=True, track_running_stats=True)
  (relu): PReLU(num_parameters=1)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): PReLU(num_parameters=1)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (1): BatchN

# 学習の実行

In [26]:
#Cross Validationを行いたい...
#Early Stopping を行いたい

num_epochs = 50
train_loss_list = []
train_acc_list = []
val_loss_list = []
val_acc_list = []

start = time.time()
for epoch in range(num_epochs):
    train_loss = 0
    train_acc = 0
    val_loss = 0
    val_acc = 0
    
    #train
    net.train()
    for i, (images, labels) in enumerate(train_loader):
        #view()での変換をしない
        images, labels = images.to(device), labels.to(device).long()
        
        features = net(images)
        outputs = metric_fc(features, labels)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        iters = epoch * len(train_loader) + i

        if iters % opt.print_freq == 0:
            outputs = outputs.data.cpu().numpy()
            outputs = np.argmax(outputs, axis=1)
            labels = labels.data.cpu().numpy()
            # print(output)
            # print(label)
            acc = np.mean((outputs == labels).astype(int))
            speed = opt.print_freq / (time.time() - start)
            time_str = time.asctime(time.localtime(time.time()))
            print('{} train epoch {} iter {} {} iters/s loss {} acc {}'.format(time_str, epoch, i, speed, loss.item(), acc))
            if opt.display:
                visualizer = Visualizer()
                visualizer.display_current_results(iters, loss.item(), name='train_loss')
                visualizer.display_current_results(iters, acc, name='train_acc')

            start = time.time()
            
#         train_loss += loss.item()
# #         train_acc += (outputs.max(1)[1]==labels).sum().item()
#         outputs = outputs.data.cpu().numpy()
#         outputs = np.argmax(outputs, axis=1)
#         labels = labels.data.cpu().numpy()
#         acc = np.mean((outputs==labels).astype(int))
#         train_acc += acc
        
#     avg_train_loss = train_loss/len(train_loader.dataset)
#     avg_train_acc = train_acc/len(train_loader.dataset)
    
#     #validation
#     net.eval()
#     with torch.no_grad():
#         for images, labels in val_loader:
#             #view()での変換をしない
#             images = images.to(device)
#             labels = labels.to(device).long()
#             features = net(images)
#             outputs = metric_fc(features, labels)
#             loss = criterion(outputs, labels)
#             val_loss += loss.item()
# #             val_acc += (outputs.max(1)[1]==labels).sum().item()
#             outputs = outputs.data.cpu().numpy()
#             outputs = np.argmax(outputs, axis=1)
#             labels = labels.data.cpu().numpy()
#             print(outputs==labels)
#             acc = np.mean((outputs==labels).astype(int))
#             val_acc += acc
#     avg_val_loss = val_loss/len(val_loader.dataset)
#     avg_val_acc = val_acc/len(val_loader.dataset)
    
#     print('Epoch [{}/{}], Loss: {loss:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}'.format(epoch+1, num_epochs, i+1, loss=avg_train_loss, val_loss=avg_val_loss, val_acc=avg_val_acc))
#     train_loss_list.append(avg_train_loss)
#     train_acc_list.append(avg_train_acc)
#     val_loss_list.append(avg_val_loss)
#     val_acc_list.append(avg_val_acc)

ValueError: expected 4D input (got 2D input)

# train, validationのloss acc のグラフを作成

In [ ]:
plt.figure()
plt.plot(range(num_epochs), train_loss_list, color='blue', linestyle='-', label='train_loss')
plt.plot(range(num_epochs), val_loss_list, color='green', linestyle='--', label='val_loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Training and validation loss')
plt.grid()

plt.figure()
plt.plot(range(num_epochs), train_acc_list, color='blue', linestyle='-', label='train_acc')
plt.plot(range(num_epochs), val_acc_list, color='green', linestyle='--', label='test_acc')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.title('Training and validation acc')
plt.grid()

# Confusion Matrix

In [ ]:
net = net.eval()

In [ ]:
pred = []
Y = []
for i, (x,y) in enumerate(test_loader):
    with torch.no_grad():
        x = x.to(device)
        y = y.to(device)
        features = net(x)
        output = metric_fc(features,y)
    pred += [int(l.argmax()) for l in output]
    Y += [int(l) for l in y]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns

In [ ]:
cmx = confusion_matrix(Y, pred)
plt.figure(figsize=(12,12))
sns.heatmap(cmx, annot=True)
plt.show()
print('accuracy: {}'.format(accuracy_score(Y, pred)))

In [ ]:
print(classification_report(Y, pred))

In [ ]:
print(cor_table)

# Heatmap

In [ ]:
from bagnets.utils import plot_heatmap, generate_heatmap_pytorch

In [ ]:
pred = []
Y = []
for i, (x,y) in enumerate(heatmap_loader):
    with torch.no_grad():
        x = x.to(device)
        y = y.to(device)
        features = net(x)
        output = metric_fc(features,y)
    pred += [int(l.argmax()) for l in output]
    Y += [int(y)]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns

In [ ]:
cmx = confusion_matrix(Y, pred, labels=cor_table.index)
plt.figure(figsize=(12,12))
sns.heatmap(pd.DataFrame(cmx, columns=cor_table.index, index=cor_table.index), annot=True)
plt.show()
print('accuracy: {}'.format(accuracy_score(Y, pred)))

In [ ]:
pd.set_option("display.max_columns",50)
pd.DataFrame({'Y':Y, 'pred':pred}).T

In [ ]:
print(classification_report(Y, pred))

In [ ]:
cor_table

In [ ]:
net.eval()

In [ ]:
def show_heatmap(net, heatmap_dataframe):
    nps = np.array(heatmap_dataframe)
    for row in nps:
        with open(row[0], 'rb') as f:
            image = Image.open(f)
            image = image.convert('RGB')
            image = image.resize((224,224))
            image = transforms.Compose([
                transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])(image)
            image = torch.unsqueeze(image,0)
            original, label = image, [row[1]]
            # torch.Tensor->np.arrayに変換
            # generate_heatmap_pytorchにはnp.arrayの入力が必要
            original = original.numpy()

        heatmap = generate_heatmap_pytorch(net, original, label, 33)

        fig = plt.figure(figsize=(8,4))
        original_image = original[0].transpose([1,2,0])
        ax = plt.subplot(121)
        ax.set_title('original')
        plt.imshow(original_image)
        plt.axis('off')

        ax = plt.subplot(122)
        ax.set_title('heatmap')
        plot_heatmap(heatmap, original_image, ax, dilation=0.5, percentile=99, alpha=.25)
        plt.axis('off')
        plt.show()

In [ ]:
show_heatmap(net, heatmap_df)

In [ ]:
heatmap_df

In [ ]:
torch.save(net.state_dict(),'/home/tamaru/scene_categorize/main/data/bagnet-pytorch.pth')

# 重みのロード方法
net = TheModelClass(*args, **kwargs)
net.load_state_dict(torch.load(PATH))